<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [7]</a>'.</span>

In [1]:
from IPython import get_ipython
from IPython.core.display import display
from typing_extensions import runtime
import numpy as np
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')
get_ipython().run_line_magic('run', 'setup')

# sorteert de kolommen alfabetisch, is makkelijker visueel te debuggen.
def sortcolumns(df):
  return df[sorted(df.columns)]

# prepareer een RIVM dataset
def prepare(dataset, day=0):
  df = RIVM.csv(dataset, day)
  # hernoem kolommen voor makkelijker uniforme data bewerking
  for old, new in [('Municipality_code', 'GemeenteCode'), ('Security_region_code', 'VeiligheidsregioCode'), ('Security_region_name', 'Veiligheidsregio')]:
    if old in df:
      df[new] = df[old]
  if 'GemeenteCode' in df:
    df['GemeenteCode'] = df['GemeenteCode'].fillna('GM0000')

  if 'Agegroup' in df:
    df['LeeftijdCode'] = 'LE' + df['Agegroup'].replace({'0-9': '00-09', '<50': '00-00', 'Unknown': '00-00', 'Onbekend': '00-00'}).replace('-', '', regex=True).astype(str)
    df['Total_reported'] = 1 # impliciet in casus-landelijk
    df = df.replace({'Hospital_admission': {'Yes': 1, 'No': 0, 'Unknown': 0}, 'Deceased': {'Yes': 1, 'No': 0, 'Unknown': 0}})

  # voeg regiocodes to aan elke regel in de dataset
  if 'GemeenteCode' in df:
    for regiotype in ['GGDregio', 'Provincie', 'Landsdeel', 'Schoolregio']:
      df = df.merge(gemeenten[['GemeenteCode', f'{regiotype}Code']].drop_duplicates(), on='GemeenteCode')

  # als er geen gemeentecode is, maar misschien wel een VR code, vervang die door VR00
  if 'GemeenteCode' in df and 'VeiligheidsregioCode' in df:
    df.loc[df.GemeenteCode == 'GM0000', 'VeiligheidsregioCode'] = 'VR00'
    df.loc[df.GemeenteCode == 'GM0000', 'Veiligheidsregio'] = ''

  df['LandCode'] = 'NL'
  df['Land'] = 'Nederland'

  # knip de tijd van de datum af, en stop hem in 'Today' (referentiepunt metingen)
  if 'Date_of_report' in df:
    df['Datum'] = df.Date_of_report.str.replace(' .*', '', regex=True)
  elif 'Date_file' in df:
    df['Datum'] = df.Date_file.str.replace(' .*', '', regex=True)
  df['Today'] = pd.to_datetime(df.Datum)

  # zet 'Date' naar de bij de betreffende dataset horende meetdatum-kolom
  for when in ['Date_statistics', 'Date_of_statistics', 'Date_of_publication']:
    if when in df:
      df['Date'] = pd.to_datetime(df[when])
      # en direct maar weken terug, die hebben we vaker nodig
      df['WekenTerug'] = ((df.Today - df.Date) / np.timedelta64(7, 'D')).astype(int)

  return sortcolumns(df).sort_values(by=['Date'])

async def publish(df, objectName):
  #df2 = df.set_index('Code')
  m = (df == np.inf)
  df2 = df.loc[m.any(axis=1), m.any(axis=0)]
  display(df2.head())

  os.makedirs('artifacts', exist_ok = True)
  df.to_csv(f'artifacts/{objectName}.csv', index=True)

  if knack:
    print('updating knack')
    await knack.update(objectName=objectName, df=df, slack=Munch(msg='\n'.join(Cache.actions), emoji=None))
    await knack.timestamps(objectName, Cache.timestamps)

In [2]:
@run
def cell():
  global gemeenten
  # rename de kolommen naar "Naam" + "NaamCode" voor makkelijker uniforme data bewerking
  gemeenten = pd.read_csv('gemeenten.csv').rename(columns={
    'Code': 'GemeenteCode',
    'Naam': 'Gemeente',
    'Veiligheidsregio Code': 'VeiligheidsregioCode',
    'GGD regio': 'GGDregio',
    'Landcode': 'LandCode',
  })
  # niet nodig want die voegen we vanzelf toe bij de per-type constructie van de cijfers
  del gemeenten['Type']
  
  global leeftijdsgroepen
  leeftijdsgroepen = pd.read_csv('leeftijdsgroepen.csv')
  del leeftijdsgroepen['Type']
  lgb = CBS.bevolking(leeftijdsgroepen=True).reset_index()
  lgb['Code'] = 'LE' + lgb['Range'].replace({'0-9': '00-09'}).replace('-', '', regex=True).astype(str)
  lgb = lgb.rename(columns={'BevolkingOpDeEersteVanDeMaand': 'Personen'})
  leeftijdsgroepen = leeftijdsgroepen.merge(lgb[['Code', 'Personen']], how='left', on='Code')

  global regiocodes
  regiocodes = pd.read_csv('regiocodes.csv')
  # sluit aan bij de uniforme naamgeving van hierboven
  regiocodes = regiocodes.rename(columns={'Landcode': 'LandCode'})
  regiocodes.loc[regiocodes.Type == 'GGD', 'Type'] = 'GGDregio'
  
  # voeg de regiocodes toe aan de gemeenten tabel voor makkelijker uniforme data bewerking
  for regiotype in ['GGDregio', 'Provincie', 'Landsdeel', 'Schoolregio']:
    gemeenten = gemeenten.merge(
      regiocodes[regiocodes.Type == regiotype][['LandCode', 'Regio', 'Code']].rename(columns={'Code': regiotype + 'Code', 'Regio': regiotype}),
      how='left',
      on=[regiotype, 'LandCode'],
    )
  gemeenten = gemeenten.merge(
    regiocodes[regiocodes.Type == 'Land'][['LandCode', 'Regio']].rename(columns={'Regio': 'Land'}),
    how='left',
    on='LandCode'
  )

  # lege regel voor GM0000
  for regiotype, prefix in [('GGDregio', 'GG'), ('Veiligheidsregio', 'VR'), ('Provincie', 'PV'), ('Landsdeel', 'LD'), ('Schoolregio', 'SR')]:
    gemeenten.loc[gemeenten.GemeenteCode == 'GM0000', regiotype] = ''
    gemeenten.loc[gemeenten.GemeenteCode == 'GM0000', f'{regiotype}Code'] = f'{prefix}00'
  gemeenten.loc[gemeenten.GemeenteCode == 'GM0000', 'LandCode'] = 'NL'
  
  base = 'https://opendata.cbs.nl/ODataApi/OData/37230ned'
  
  # voor perioden pak de laatste
  periode = CBS.odata(base + '/Perioden').iloc[[-1]]['Key'].values[0]
  
  # startsWith would have been better to do in the filter but then the CBS "odata-ish" server responds with
  # "Object reference not set to an instance of an object."
  bevolking = CBS.odata(base + f"/TypedDataSet?$filter=(Perioden eq '{periode}')&$select=RegioS, BevolkingAanHetBeginVanDePeriode_1")
  # want de CBS odata API snap startsWith niet...
  bevolking = bevolking[bevolking.RegioS.str.startswith('GM')]
  # die _1 betekent waarschijnlijk dat het gedrag ooit gewijzigd is en er een nieuwe "versie" van die kolom is gepubliceerd
  bevolking.rename(columns={'RegioS': 'GemeenteCode', 'BevolkingAanHetBeginVanDePeriode_1': 'BevolkingAanHetBeginVanDePeriode'}, inplace=True)
  
  gemeenten = gemeenten.merge(bevolking, how='left', on='GemeenteCode')
  # default naar gegeven waarde bij ontbrekende data
  gemeenten.loc[gemeenten.Personen == 0, 'Personen'] = gemeenten.BevolkingAanHetBeginVanDePeriode
  del gemeenten['BevolkingAanHetBeginVanDePeriode']
  gemeenten = sortcolumns(gemeenten)

  global aantallen, ziekenhuisopnames, ziekenhuisopnames_1, casus_landelijk, casus_landelijk_1
  aantallen = prepare('COVID-19_aantallen_gemeente_per_dag', 0)
  ziekenhuisopnames = prepare('COVID-19_ziekenhuisopnames', 0)
  ziekenhuisopnames_1 = prepare('COVID-19_ziekenhuisopnames', 1)
  casus_landelijk = prepare('COVID-19_casus_landelijk', 0)
  casus_landelijk_1 = prepare('COVID-19_casus_landelijk', 1)

downloading rivm/COVID-19_aantallen_gemeente_per_dag-2021-02-24@14-15.csv


rivm COVID-19_aantallen_gemeente_per_dag zipping rivm/COVID-19_aantallen_gemeente_per_dag-2021-02-24@14-15.csv


loading rivm/COVID-19_aantallen_gemeente_per_dag-2021-02-24@14-15.csv.gz


downloading rivm/COVID-19_ziekenhuisopnames-2021-02-24@14-15.csv


rivm COVID-19_ziekenhuisopnames zipping rivm/COVID-19_ziekenhuisopnames-2021-02-24@14-15.csv


loading rivm/COVID-19_ziekenhuisopnames-2021-02-24@14-15.csv.gz


loading rivm/COVID-19_ziekenhuisopnames-2021-02-23@14-15.csv.gz


rivm/COVID-19_casus_landelijk-2021-02-24@14-15.csv exists
loading rivm/COVID-19_casus_landelijk-2021-02-24@14-15.csv.gz


loading rivm/COVID-19_casus_landelijk-2021-02-23@14-15.csv.gz


In [3]:
def groupregio(regiotype):
  """
    Groepeer de gemeenten tabel op gegeven regiotypen en sommeer personen/oppervlakte
  """
  global gemeenten

  grouping = [ f'{regiotype}Code', regiotype]
  if regiotype != 'Land':
    grouping += [ 'LandCode' ]

  # deze kolommen willen we voor de resultset, ook al zijn ze voor alles behalve 'Gemeente' leeg
  columns = [
    'GGDregio',
    'Veiligheidsregio',
    'VeiligheidsregioCode',
    'Provincie',
    'Landsdeel',
    'Schoolregio',
    'Land',
  ]

  if regiotype == 'Gemeente':
    # hier hoeven we niets te doen dan de juiste kolommen te selecteren
    df = gemeenten[grouping + columns + ['Personen', 'Opp land km2']].rename(columns={'Gemeente': 'Naam', 'GemeenteCode': 'Code'})
  elif regiotype == 'Leeftijd':
    df = (leeftijdsgroepen
      # voeg de lege kolommen toe
      .assign(**{ col: '' for col in columns})
      .assign(**{'Opp land km2': 0})
    )
  else:
    df = (gemeenten[gemeenten.GemeenteCode != 'GM0000']
      # groupeer op regiotype, sommeer oppervlakte en personen
      .groupby(grouping).agg({ 'Personen': 'sum', 'Opp land km2': 'sum' })
      .reset_index()
      # standardiseerd de 'Naam' en 'Code' kolommen zodat ze klaar zijn voor output.
      .rename(columns={regiotype: 'Naam', f'{regiotype}Code': 'Code'})
      # voeg de lege kolommen toe
      .assign(**{ col: '' for col in columns })
    )
    if regiotype == 'Land':
      df['Land'] = df['Naam']
      df['LandCode'] = df['Code']
  # voeg het regiotype toe in de Type kolom
  return df.assign(Type=regiotype)

def sumcols(df, regiotype, columns):
  """
    groepeer en sommeer deceased/admission/positive
  """
  regiotype_code = f'{regiotype}Code'
  return (df
    # groepeer op regiotype en selecteer de gewenste kolommen
    .groupby([regiotype_code])[list(columns.keys())]
    # sommeer
    .sum()
    # rename naar gewenste output kolommen
    .rename(columns=columns)
  )

def histcols(df, regiotype, maxweeks, column, colors=False, highestweek=False):
  """
    voeg week historie toe:
    - regiotype
    - maxweeks = hoeveel weken
    - column = deceased/admission/positive naam => output kolom
    - colors = toevoegen schaalverdeling kleuren
    - highestweek = toevoegen op welke week het maximum was behaald
  """
  # in principe zouden we kunnen groeperen/sommeren op meerdere kolommen tegelijk maar dan worden colors en highestweek weer heel complex
  assert len(column) == 1
  label = list(column.values())[0]
  datacolumn = list(column.keys())[0]
  regiotype_code = f'{regiotype}Code'

  # knip alle data van >= maxweeks eruit
  df = df[df.WekenTerug < maxweeks]

  # als we schalen naar 100.000, voor hoeveel telt elke melding dan mee
  if 'scale' in df:
    df = df.assign(count=df[datacolumn] * df.scale).replace(np.inf, 0)
  else:
    df = df.assign(count=df[datacolumn])

  df = (df
    # groepeer op reguitype en wekenterug
    .groupby([regiotype_code, 'WekenTerug'])['count']
    # optellen (de aantallen zijn eventueel hierboven al geschaald)
    .sum()
    # maak de week nummers kolommen
    .unstack(level=-1)
    # en vul de kolommen uit zodat als een week helemaal geen meldingen heeft dat die niet weg blijft maar 0 bevat
    .reindex(columns=np.arange(maxweeks), fill_value=0)
  )

  merges = []
  # must be done before colors is merged and before the columns are renamed
  if highestweek:
    merges.append(df.idxmax(axis=1).to_frame().rename(columns={0: f'{label} hoogste week'}))

  # hernoem de kolommen van de getallen die ze nu zijn
  df.columns = [f'{label} w{-col}' for col in df.columns.values]

  # must be done before highestweek is merged but after the columns are renamed
  if colors:
    # kleurkolommen zijn waarde van de week gedeeld door het maximum over de weken heen
    merges.append((df.divide(df.max(axis=1), axis=0) * 1000).rename(columns={col:col.replace(' w', ' cw') for col in df}))

  for extra in merges:
    df = df.merge(extra, left_index=True, right_index=True)

  # bij ontbreken van w-1 vaste waarde 9.99
  df[f'{label} t.o.v. vorige week'] = (df[f'{label} w0'] / df[f'{label} w-1']).replace(np.inf, 9.99)
  return df

def collect(regiotype):
  """
    verzamel alle kolommen voor gegeven regiotype
  """
  regiotype_code = f'{regiotype}Code'

  def datasets():
    if regiotype == 'Leeftijd':
      global casus_landelijk, casus_landelijk_1
      return (casus_landelijk, casus_landelijk, casus_landelijk_1)
    else:
      global aantallen, ziekenhuisopnames, ziekenhuisopnames_1
      return (aantallen, ziekenhuisopnames, ziekenhuisopnames_1)

  aantallen, ziekenhuisopnames, ziekenhuisopnames_1 = datasets()

  assert len(aantallen.Datum.unique()) == 1
  assert len(ziekenhuisopnames_1.Datum.unique()) == 1
  assert list(aantallen.Datum.unique()) == list(ziekenhuisopnames.Datum.unique()), list(aantallen.Datum.unique()) + list(ziekenhuisopnames.Datum.unique())

  # sommeer Total_reported en Deceased voor gegeven regiotype
  pos_dec = sumcols(aantallen, regiotype, {'Total_reported':'Positief getest', 'Deceased':'Overleden'})
  # toename is precies hetzelfde maar dan alleen voor de metingen van 'vandaag'
  pos_dec_delta = sumcols(
    aantallen[aantallen.Date == aantallen.Today],
    regiotype,
    {'Total_reported':'Positief getest (toename)', 'Deceased':'Overleden (toename)'}
  )
  # sommeer Hospital_admission voor gegeven regiotype
  admissions = sumcols(ziekenhuisopnames, regiotype, {'Hospital_admission':'Ziekenhuisopname'})
  # sommeer Hospital_admission van 'vorige dag' voor gegeven regiotype
  admissions_1 = sumcols(ziekenhuisopnames_1, regiotype, {'Hospital_admission':'Ziekenhuisopname_1'})
  # voeg ze bij elkaar en trek ze van elkaar af
  admissions_delta = admissions.merge(admissions_1, how='left', on=regiotype_code)
  admissions_delta['Ziekenhuisopname (toename)'] = admissions_delta.Ziekenhuisopname - admissions_delta.Ziekenhuisopname_1
  # niet meer nodig en vervuilen anders het resultaat
  del admissions_delta['Ziekenhuisopname']
  del admissions_delta['Ziekenhuisopname_1']

  # groupeer de gemeenten op gegeven regiotype
  df = (groupregio(regiotype)
    # en voeg de berekende kolommen toe
    .merge(pos_dec, how='left', left_on='Code', right_index=True)
    .merge(admissions, how='left', left_on='Code', right_index=True)
    .merge(pos_dec_delta, how='left', left_on='Code', right_index=True)
    .merge(admissions_delta, how='left', left_on='Code', right_index=True)
    .fillna(0)
  )
  # per 100k voor de absolute kolommen
  for cat in [pos_dec, admissions]:
    for col in cat.columns:
      df[col + ' per 100.000'] = ((df[col] * 100000) / df.Personen).replace(np.inf, 0)

  df['Positief getest 1d/100k'] = ((df['Positief getest (toename)'] * 100000) / df.Personen).replace(np.inf, 0)
  df['Positief getest percentage'] = (df['Positief getest'] / df.Personen).replace(np.inf, 0)
  df['Positief getest per km2'] = (df['Positief getest'] / df['Opp land km2']).replace(np.inf, 0)

  # weekhistories
  maxweeks = 26
  df = (df
    .merge(histcols(
      aantallen,
      regiotype,
      maxweeks=maxweeks,
      colors=True,
      highestweek=True,
      column={'Total_reported':'Positief getest'}), how='left', left_on='Code', right_index=True)
    .merge(histcols(
      aantallen.merge(df.assign(scale=100000 / df.Personen)[['Code', 'scale']], left_on=regiotype_code, right_on='Code'),
      regiotype,
      maxweeks=maxweeks,
      column={'Total_reported':'Positief getest 7d/100k'}), how='left', left_on='Code', right_index=True)
    .merge(histcols(ziekenhuisopnames,
      regiotype,
      maxweeks=maxweeks,
      colors=True,
      column={'Hospital_admission':'Ziekenhuisopname'}), how='left', left_on='Code', right_index=True)
    .merge(histcols(
      aantallen,
      regiotype,
      maxweeks=maxweeks,
      colors=True,
      column={'Deceased':'Overleden'}), how='left', left_on='Code', right_index=True)
  )
  df['Datum'] = aantallen.Datum.unique()[0]

  return df

In [4]:
@run
def cell():
  global regios
  # verzamel de data voor de gegeven regiotypes en plak ze onder elkaar
  regios = pd.concat([
    collect(regiotype)
    for regiotype in
    [
      'Gemeente',
      'GGDregio',
      'Veiligheidsregio',
      'Provincie',
      'Landsdeel',
      'Schoolregio',
      'Land',
      'Leeftijd',
    ]
  ])
  # maak de kolommen leeg voor GM0000
  # hernoem de eerder geuniformeerde kolomen terug naar de gewenste output
  regios = regios.rename(columns={
    'LandCode': 'Landcode',
    'VeiligheidsregioCode': 'Veiligheidsregio Code',
    'GGDregio': 'GGD regio'
  })

In [5]:
order = pd.read_csv('RegioV2.csv')
await publish(regios[order.columns.values].fillna(0), 'RegioV2')

""


updating knack


  0%|          | 0/434 [00:00<?, ?it/s]

  0%|          | 1/434 [00:00<07:01,  1.03it/s]

  1%|          | 4/434 [00:01<01:38,  4.38it/s]

...


rate limit: 4 
API calls: create: 0, read: 2, update: 434, delete: 0, backoff: 0
errors:
  503: Service Unavailable: 3


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:01<00:00,  1.04s/it]

100%|██████████| 1/1 [00:01<00:00,  1.04s/it]


rate limit: 4 
API calls: create: 0, read: 1, update: 1, delete: 0, backoff: 0


In [6]:
# just so the timestamps update OK
Cache.reset()
aantallen = prepare('COVID-19_aantallen_gemeente_per_dag')
ziekenhuisopnames = prepare('COVID-19_ziekenhuisopnames')
@run
def cell():
  import sys
  global aantallen, ziekenhuisopnames
  aantallen['Week'] = aantallen['Date'].dt.strftime('%Y-%U')
  ziekenhuisopnames['Week'] = ziekenhuisopnames['Date'].dt.strftime('%Y-%U')

  weken = list(pd.date_range(
    start=min(aantallen.Date.min(), ziekenhuisopnames.Date.min()),
    end=max(aantallen.Date.max(), ziekenhuisopnames.Date.max())
  ).strftime('%Y-%U').unique())
  print(len(weken), 'weken')

  regiotypes = [ 'GGDregio', 'Gemeente', 'Land', 'Landsdeel', 'Provincie', 'Schoolregio', 'Veiligheidsregio' ]
  for regiotype in regiotypes:
    assert regiotype + 'Code' in aantallen, (regiotype, aantallen.columns)
    assert regiotype + 'Code' in ziekenhuisopnames, (regiotype, ziekenhuisopnames.columns)

  global regioposten
  regioposten = []
  for regiotype in regiotypes:
    print(regiotype)
    sys.stdout.flush()
    code = regiotype + 'Code'

    ag = sortcolumns(
      aantallen[['Week', code, 'Date', 'Total_reported', 'Deceased']]
      .assign(Hospital_admission=np.nan)
    )
    zo = sortcolumns(
      ziekenhuisopnames[['Week', code, 'Date', 'Hospital_admission']]
      .assign(Total_reported=np.nan, Deceased=np.nan)
    )
    df = (pd.concat([ag, zo], axis=0)
      .sort_values(by=['Date'])
      .groupby(['Week', code])
      .agg({
        'Date': [ 'max', 'nunique' ],
        'Total_reported': [ 'sum', 'last' ],
        'Deceased': [ 'sum', 'last' ],
        'Hospital_admission': [ 'sum', 'last' ],
      })
      .reset_index()
    )
    df.columns = [' '.join(col).strip() for col in df.columns.values]
    df.rename(columns={
      code: 'Code',
      'Date max': 'Datum',
      'Date nunique': 'Dagen',
      'Total_reported sum': 'Positief getest week',
      'Total_reported last': 'Positief getest laatste dag',
      'Deceased sum': 'Overleden week',
      'Deceased last': 'Overleden laatste dag',
      'Hospital_admission sum': 'Ziekenhuisopname week',
      'Hospital_admission last': 'Ziekenhuisopname laatste dag',
    }, inplace=True)

    regio = groupregio(regiotype)
    regio = regio[[col for col in regio.columns if col == 'Code' or 'Code' not in col and 'km2' not in col]]
    df = df.merge(regio, how='left', on='Code')
    for col, coltype in zip(regio.columns, regio.dtypes):
      if col == 'Personen':
        df[col] = df[col].fillna(0).astype(int)
      elif col == 'Type':
        df[col] = df[col].fillna(regiotype)
      elif coltype == np.float64:
        df[col] = df[col].fillna(0)
      elif coltype == object:
        df[col] = df[col].fillna('')
      else:
        raise KeyError(col)
    df['Landcode'] = 'NL'

    base = [
      (code, week)
      for code in list(df.Code.unique())
      for week in weken
    ]
    base = pd.DataFrame({
        'Week': [week for code, week in base],
        'Code': [code for code, week in base],
      },
      index=['-'.join(codeweek) for codeweek in base]
    )

    df['Key'] = df['Code'] + '-' + df['Week']
    df.set_index('Key', inplace=True)
    df.drop(columns=['Code', 'Week'], inplace=True)

    df = base.join(df)

    for col in ['Positief getest', 'Overleden', 'Ziekenhuisopname']:
      # vul missende waarden met 0 vanaf de eerstgevonden waarde zodat cumsum goed werkt
      df[f'{col} week'] = df[f'{col} week'].mask(df[f'{col} week'].ffill().isnull(), 0)
      df[f'{col} cumulatief'] = df[f'{col} week'].cumsum().fillna(0)
      df[f'{col} week -1'] = df[f'{col} cumulatief'].shift(1).fillna(0)
      df[f'{col} week'] = df[f'{col} week'].fillna(0)

    df.index.rename('Key', inplace=True)
    df = df[[col for col in df.columns if col != 'Land']]
    regioposten.append(df.reset_index())
  regioposten = pd.concat(regioposten, axis=0)
  regioposten['Datum'] = regioposten['Datum'].dt.strftime('%Y-%m-d %H:%M')
  regioposten.rename(columns={'GGDregio': 'GGD regio'}, inplace=True)
  display(regioposten)

rivm/COVID-19_aantallen_gemeente_per_dag-2021-02-24@14-15.csv exists
loading rivm/COVID-19_aantallen_gemeente_per_dag-2021-02-24@14-15.csv.gz


rivm/COVID-19_ziekenhuisopnames-2021-02-24@14-15.csv exists
loading rivm/COVID-19_ziekenhuisopnames-2021-02-24@14-15.csv.gz


54 weken
GGDregio


Gemeente


Land


Landsdeel


Provincie


Schoolregio


Veiligheidsregio


,Key,Week,Code,Datum,Dagen,Positief getest week,Positief getest laatste dag,Overleden week,Overleden laatste dag,Ziekenhuisopname week,...,Landsdeel,Schoolregio,Type,Landcode,Positief getest cumulatief,Positief getest week -1,Overleden cumulatief,Overleden week -1,Ziekenhuisopname cumulatief,Ziekenhuisopname week -1
0,GG00-2020-08,2020-08,GG00,2020-02-d 00:00,3,0.0,0.0,0.0,0.0,0.0,...,,,GGDregio,NL,0.0,0.0,0.0,0.0,0.0,0.0
1,GG00-2020-09,2020-09,GG00,2020-03-d 00:00,7,0.0,0.0,0.0,0.0,2.0,...,,,GGDregio,NL,0.0,0.0,0.0,0.0,2.0,0.0
2,GG00-2020-10,2020-10,GG00,2020-03-d 00:00,7,5.0,0.0,0.0,0.0,18.0,...,,,GGDregio,NL,5.0,0.0,0.0,0.0,20.0,2.0
3,GG00-2020-11,2020-11,GG00,2020-03-d 00:00,7,27.0,1.0,0.0,0.0,32.0,...,,,GGDregio,NL,32.0,5.0,0.0,0.0,52.0,20.0
4,GG00-2020-12,2020-12,GG00,2020-03-d 00:00,7,33.0,1.0,1.0,0.0,72.0,...,,,GGDregio,NL,65.0,32.0,1.0,0.0,124.0,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399,VR25-2021-04,2021-04,VR25,2021-01-d 00:00,7,664.0,11.0,5.0,1.0,12.0,...,,,Veiligheidsregio,NL,1067207.0,1066543.0,15390.0,15385.0,45600.0,45588.0
1400,VR25-2021-05,2021-05,VR25,2021-02-d 00:00,7,547.0,7.0,4.0,0.0,16.0,...,,,Veiligheidsregio,NL,1067754.0,1067207.0,15394.0,15390.0,45616.0,45600.0
1401,VR25-2021-06,2021-06,VR25,2021-02-d 00:00,7,463.0,13.0,3.0,0.0,8.0,...,,,Veiligheidsregio,NL,1068217.0,1067754.0,15397.0,15394.0,45624.0,45616.0
1402,VR25-2021-07,2021-07,VR25,2021-02-d 00:00,7,452.0,12.0,6.0,0.0,9.0,...,,,Veiligheidsregio,NL,1068669.0,1068217.0,15403.0,15397.0,45633.0,45624.0


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [7]:
await publish(regioposten, 'Regioposten')

""


updating knack


  0%|          | 0/22949 [00:00<?, ?it/s]

  0%|          | 1/22949 [00:11<74:55:28, 11.75s/it]

  0%|          | 2/22949 [00:12<33:57:05,  5.33s/it]

...

 13%|█▎        | 2921/22949 [12:29<1:58:17,  2.82it/s]

 13%|█▎        | 2926/22949 [12:30<1:33:26,  3.57it/s]

update
Munch({'field_1809': 'LD00-2020-44', 'field_1810': '2020-44', 'field_1815': 'LD00', 'field_1811': '2020-11-d 00:00', 'field_1812': 7, 'field_1824': 179.0, 'field_1826': 4.0, 'field_1832': 1.0, 'field_1834': 0.0, 'field_1828': 55.0, 'field_1830': 4.0, 'field_1836': '', 'field_1816': 0, 'field_1818': '', 'field_1819': '', 'field_1817': '', 'field_1820': '', 'field_1821': '', 'field_1813': 'Landsdeel', 'field_1837': '5f0dfbfb80347906bdc450a6', 'field_1823': 1633.0, 'field_1825': 1454.0, 'field_1831': 23.0, 'field_1833': 22.0, 'field_1827': 676.0, 'field_1829': 621.0, 'field_1835': '54b11992ec9c0136658265793f80d15a64fa638c3121a0a53e5141d1ebf936e4'})


 13%|█▎        | 2929/22949 [12:31<1:27:34,  3.81it/s]

ClientResponseError: 503, message='Service Unavailable', url=URL('https://api.knack.com/v1/objects/object_60/records/60353210099928001b014602')